## Langchain 연결하기

In [1]:
import os, openai
from openai import OpenAI
from langchain.tools import Tool
from langchain.chains import SequentialChain, LLMChain
from langchain.chains.base import Chain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from googleapiclient.discovery import build

from typing import Dict
from typing import Dict
from dotenv import load_dotenv

In [2]:
import sys
sys.path.append('../python')
from utils import is_pdf_by_signature, encode_image, pdf_to_image, extract_json_from_string

In [3]:
# 환경 변수에서 API 키 로드
load_dotenv('../.env')

GOOGLE_API_KEY = os.getenv("GOOGLE-SEARCH-API-KEY")
SEARCH_ENGINE_ID = os.getenv("SEARCH-ENGINE-ID")
openai.api_key = os.getenv("OPEN-AI")
client = OpenAI(api_key = openai.api_key)

### OCR Chain

In [4]:
class OCRChain(Chain):
    """
    OpenAI를 이용해서 OCR을 진행하는 사용자 정의 체인.
    """
    def __init__(self):
        super().__init__()

    @property
    def input_keys(self):
        return ["image_path"]

    @property
    def output_keys(self):
        return ["ocr_response"]  # 출력값으로 Assistant의 응답을 반환

    def _call(self, inputs: Dict[str, str]) -> Dict[str, str]:
        image_path = inputs["image_path"]
        # print(f"Image path: {image_path}")
        try:
            # 이미지 파일 열기
            if is_pdf_by_signature(image_path):
                base64_image = pdf_to_image(image_path)
            else:
                base64_image = encode_image(image_path)
            response = openai.chat.completions.create(
                model="gpt-4o",  # GPT-4 Vision 모델 사용
                messages=[
                    {
                        "role": "system", 
                        "content": "You are an assistant that extracts information from receipt images."
                    },
                    {
                        "role": "user", 
                        "content": [
                            {
                                "type": "text",
                                "text": '''다음 텍스트는 영수증의 정보입니다. 이 텍스트에서 가게 이름, 날짜, 항목, 총액을 분석하고, 아래의 JSON형식으로 결과를 반환해 주세요.
                                JSON 형식:
                                {
                                    "상호명": "가게 이름",
                                    "날짜": "YYYY-MM-DD",
                                    "항목": [
                                        {"이름": "상품1", "가격": 상품1 가격},
                                        {"이름": "상품2", "가격": 상품2 가격}
                                    ],
                                    "총액": 총액
                                }
                                반환할 JSON 형식은 반드시 위의 구조와 일치해야 하며, 불필요한 설명은 포함하지 마세요.
                                영수증 텍스트:
                                """
                                [여기에 영수증의 텍스트 또는 OCR로 추출한 내용이 들어갑니다]
                                """
                                결과:''' 
                            },{
                                "type": "image_url",
                                "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"},
                            }
                        ],
                    },
                ],
            )
            
            # 응답 파싱
            raw_response = extract_json_from_string(response.choices[0].message.content)
            return {"ocr_response": raw_response}
        except Exception as e:
            
            return {"ocr_response": f"Error during OpenAI API call: {e}"}

In [7]:
image_path = "../image/IMG_9258.jpg"  # 실제 이미지 파일 경로를 여기에 설정
# OCRChain 초기화
ocr_chain = OCRChain()
ocr_response = ocr_chain.invoke({"image_path": image_path})

In [8]:
ocr_response

{'image_path': '../image/IMG_9258.jpg',
 'ocr_response': {'상호명': '현대프리미엄아울렛 김포점 바버',
  '날짜': '2024-12-15',
  '항목': [{'이름': '바버/남성캐주얼', '가격': 207300}],
  '총액': 207300}}

### SearchChain

In [5]:
def search_with_google_api(query, num_results=10):
    api_key = GOOGLE_API_KEY  # Google API 키
    cse_id = SEARCH_ENGINE_ID   # Custom Search Engine ID
    try:
        service = build("customsearch", "v1", developerKey=api_key)
        results = service.cse().list(q=query, cx=cse_id, num=num_results).execute()
        print('results = ', results)
        items = results.get("items", [])
        return [
            {
                "title": item["title"], "snippet": item.get("snippet")  # 본문 내용 추가
            } 
            for item in items
            if item.get("snippet") is not None
        ]
    except Exception as e:
        return f'Google Search API Error: {e}'


class SearchChain(Chain):
    def __init__(self, tool):
        super().__init__()
        self._tool = tool

    @property
    def input_keys(self):
        return ["ocr_response"]

    @property
    def output_keys(self):
        return ["search_results", "business_name"]

    def _call(self, inputs, **kwargs):
        business_name = inputs['ocr_response']["상호명"]
        item = ','.join([x["이름"] for x in inputs['ocr_response']['항목']])
        print('business_name = ', business_name)
        print('item = ', item)
        if len(item) > 0:
            query = ','.join([business_name, item])
        else:
            query = business_name
        print('query = ', query)
        search_results = self._tool.func(query)
        return {"search_results": search_results, "business_name": business_name}

# Tool로 검색 기능 정의
search_tool = Tool(
    name = "SearchBusinessCategory",
    func = lambda business_name: "\n".join(
        list([x['snippet'] for x in search_with_google_api(business_name)])
    ),
    description="상호명을 검색하여 관련 업종 정보를 제공합니다."
)

In [9]:
# SearchChain 초기화
search_chain = SearchChain(search_tool)
search_response = search_chain.invoke(ocr_response)

### Analysis chain

In [6]:
# Prompt 정의
analyze_prompt = PromptTemplate(
    input_variables=["business_name", "search_results"],
    template="""
    다음은 상호명 '{business_name}'에 대한 검색 결과입니다:
    {search_results}
    검색 결과를 기반으로 해당 상호의 업종을 추론하세요. 가능한 경우 단답형으로 업종만 답변하세요.
    예: "식당", "카페", "의류", "교통" 등. 
    """
)

In [11]:
# 검색 결과를 분석하는 LLMChain
analysis_chain = LLMChain(
    llm = ChatOpenAI(
        model="gpt-4", 
        temperature=0,
        openai_api_key = openai.api_key
    ),
    prompt=analyze_prompt,
    output_key="business_category",  # 출력 키를 명시적으로 설정
)

/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_48104/1281889064.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(
/var/folders/39/jrqp49_54fj1tl_h3pjv45780000gn/T/ipykernel_48104/1281889064.py:5: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  analysis_chain = LLMChain(


### 전체 Chain

In [12]:
# SequentialChain 구성
sequential_chain = SequentialChain(
    chains=[ocr_chain, search_chain, analysis_chain],
    input_variables=["image_path"],
    output_variables=["business_category"],
    verbose=True
)

In [13]:
# 실행
image_path = "../image/naver.png"  # 실제 이미지 파일 경로를 여기에 설정
result = sequential_chain.invoke({"image_path": image_path})
print("\n최종 결과:")
print(result["business_category"])



> Entering new SequentialChain chain...
business_name =  네이버파이낸셜 주식회사
item =  
query =  네이버파이낸셜 주식회사
results =  {'kind': 'customsearch#search', 'url': {'type': 'application/json', 'template': 'https://www.googleapis.com/customsearch/v1?q={searchTerms}&num={count?}&start={startIndex?}&lr={language?}&safe={safe?}&cx={cx?}&sort={sort?}&filter={filter?}&gl={gl?}&cr={cr?}&googlehost={googleHost?}&c2coff={disableCnTwTranslation?}&hq={hq?}&hl={hl?}&siteSearch={siteSearch?}&siteSearchFilter={siteSearchFilter?}&exactTerms={exactTerms?}&excludeTerms={excludeTerms?}&linkSite={linkSite?}&orTerms={orTerms?}&dateRestrict={dateRestrict?}&lowRange={lowRange?}&highRange={highRange?}&searchType={searchType}&fileType={fileType?}&rights={rights?}&imgSize={imgSize?}&imgType={imgType?}&imgColorType={imgColorType?}&imgDominantColor={imgDominantColor?}&alt=json'}, 'queries': {'request': [{'title': 'Google Custom Search - 네이버파이낸셜 주식회사', 'totalResults': '57400', 'searchTerms': '네이버파이낸셜 주식회사', 'count': 10, 'st